In [1]:
%%time
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import pandas as pd
import numpy as np
import scipy.io.wavfile
from scipy import fftpack

import matplotlib.pyplot as plt
import matplotlib.style as ms
ms.use('seaborn-muted')
%matplotlib inline

from os import sep
from os.path import isfile, join, abspath, expanduser
from moviepy.editor import AudioFileClip, VideoFileClip, concatenate_videoclips, concatenate_audioclips

from sklearn.decomposition import PCA
from ggplot import *

Wall time: 4.1 s


In [2]:
home = expanduser("~")
dirpath = join(home,"Downloads","videos")
file_basename = 'MON_V_MONT_1H_ESP'
eventFileName = join(dirpath, file_basename+'.csv')
event_data = pd.read_csv(eventFileName, sep=',',header=0)
print(len(event_data),' events loaded')


7  events loaded


In [3]:
# Compute offset of each clip in the new clips collection and add to DataFrame
offset=np.cumsum(event_data["timeToCapture"])
event_data["offset"]=offset-event_data["timeToCapture"]
# Prepare a list of labels
labels=[row["eventName"] 
        for index,row in event_data.iterrows() 
        for i in np.arange(row["offset"],row["offset"]+row["timeToCapture"])]

# Concatenate video clips using videoPy


In [7]:
%%time
clips=[]
#fade_duration = 1 # 1-second fade-in for each clip
for index, row in event_data.iterrows():
    #print row['secondOffset'], row['timeToCapture']
    videoFileName = join(dirpath, row['fileName'])
    if isfile(videoFileName):
        videoClip = VideoFileClip(videoFileName).subclip(row['secondOffset'],row['secondOffset']+row['timeToCapture'])
        # clips.append(videoClip.crossfadein(fade_duration))
        clips.append(videoClip)
    else:
        print ('Could not find file ', videoFileName)
    
# final_clip = concatenate_videoclips(clips, padding = fade_duration)
final_clip = concatenate_videoclips(clips)
final_clip.write_videofile(join(dirpath, file_basename+"_highlight.mp4"))

[MoviePy] >>>> Building video C:\Users\Roy\Downloads\videos\MON_V_MONT_1H_ESP_highlight.mp4
[MoviePy] Writing audio in MON_V_MONT_1H_ESP_highlightTEMP_MPY_wvf_snd.mp3


100%|████████████████████████████████████████████████████████████████████████████| 4785/4785 [00:04<00:00, 1140.74it/s]


[MoviePy] Done.
[MoviePy] Writing video C:\Users\Roy\Downloads\videos\MON_V_MONT_1H_ESP_highlight.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 5425/5426 [01:15<00:00, 71.42it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: C:\Users\Roy\Downloads\videos\MON_V_MONT_1H_ESP_highlight.mp4 

Wall time: 1min 23s
